In [9]:
import gzip # Для работы архиватора.
from tqdm.notebook import tqdm
from math import trunc              # Для корректной работы функции бинарного поиска.
from collections import defaultdict # Для построения словаря обратного индекса.
import random                       # Для формирования id для узлов в дереве.
from copy import deepcopy           # В тех местах, где передаются списки, нужно
                                    #  передавать их копии, чтобы не изменять оригинал.

In [2]:
def clear_data(dirname):
    
    # Для того, чтобы почистить url от сиволов типа \x*
    #
    def __cleaned(dirty_url):
    
        ord_th = 46

        buf = ""

        for c in dirty_url:

            buf += c

            if len(buf) == 4:
                if buf != "http":
                    buf = buf[1:]      

            if (ord(buf[-1]) < ord_th) and ('http' in buf):
                    return buf[:-1]
        return buf[:-1]

    
    # Получаем из всех не предобработанных
    # файлов список из слов.
    #
    data = []
    for i in range(1,8+1,1):

        filename = "file_" + str(i) + ".gz"
        with gzip.open(dirname + filename) as f:
            data.extend(f.read().decode("utf-8", errors="ignore").lower().split())
            
    # Формируем словарь вида - {url: list of word}
    # И множество (set) всех слов в корпусе.
    #
    buffer = []
    cites  = dict()
    word_set = set(data)

    for word in data:

        if 'http' in word:
            if len(buffer) > 2:
                cites[__cleaned(buffer[0])] = buffer[1:]
                word_set.remove(buffer[0])
            buffer = []
        buffer.append(word)
            
    return cites, word_set

In [3]:
%%time

# Main:
#
cites, word_set = clear_data('dataset/')                      # Словарь вида {url сайта: список слов на сайте}
term_to_id = {word: i for i, word in enumerate(word_set)}     # -//- {слово: id слова} (нумерация с 0 до ~300k)
url_to_docid = {url: i for i, url in enumerate(cites.keys())} # -//- {url: id сайта} (нумерация с 0 до ~9k)
docid_to_url = {i: url for i, url in enumerate(cites.keys())} # -//- {id сайта: url}


reversed_index = defaultdict(list)                            # Словарь обратных индексов вида
for i, (url, word_list) in enumerate(cites.items()):          #  {id слова: все id сайтов, на которых оно есть}
    for word in word_list:
        reversed_index[term_to_id[word]].append(url_to_docid[url]) # Already sorted

CPU times: user 6.22 s, sys: 332 ms, total: 6.55 s
Wall time: 6.58 s


In [4]:
class Simple9_encoder_decoder():
    def __init__(self):

        # Формируем маски-константы.
        #
        code_9 = 0x90000000 # 1001 0000 0000 ... 0000
        code_8 = 0x80000000 # 1000 0000 0000 ... 0000
        code_7 = 0x70000000 # 0111 0000 0000 ... 0000
        code_6 = 0x60000000 # 0110 0000 0000 ... 0000
        code_5 = 0x50000000 # 0101 0000 0000 ... 0000
        code_4 = 0x40000000 # 0100 0000 0000 ... 0000
        code_3 = 0x30000000 # 0011 0000 0000 ... 0000
        code_2 = 0x20000000 # 0010 0000 0000 ... 0000
        code_1 = 0x10000000 # 0001 0000 0000 ... 0000
        
        self.encode_type = [[28, 2**1  - 1, code_9,  1],     
                            [14, 2**2  - 1, code_8,  2],
                            [ 9, 2**3  - 1, code_7,  3],
                            [ 7, 2**4  - 1, code_6,  4],
                            [ 5, 2**5  - 1, code_5,  5],
                            [ 4, 2**7  - 1, code_4,  7],
                            [ 3, 2**9  - 1, code_3,  9],
                            [ 2, 2**14 - 1, code_2, 14],
                            [ 1, 2**28 - 1, code_1, 28]
                           ]

        self.decode_type = {code_9: [28, 2**1  - 1,  1],     
                            code_8: [14, 2**2  - 1,  2],
                            code_7: [9,  2**3  - 1,  3],
                            code_6: [7,  2**4  - 1,  4],
                            code_5: [5,  2**5  - 1,  5],
                            code_4: [4,  2**7  - 1,  7],
                            code_3: [3,  2**9  - 1,  9],
                            code_2: [2,  2**14 - 1, 14],
                            code_1: [1,  2**28 - 1, 28]
                           }
        
    def encode(self, a):
        """
        Метод класса, позволяюзий КОДИРОВАТЬ
        список элементов a, по принципу Simple9.
        """

        offset    = 0        
        res       = []
        list_size = len(a)

        while offset < list_size:        

            for current_encode_type in self.encode_type:

                n     = current_encode_type[0] # Кол-во чисел которые могут поместиться на регистр.
                th    = current_encode_type[1] # Верхнее возможное число для выбранного code_x.
                code  = current_encode_type[2] # Запоминаем n (важны лишь первые 4 бита)
                shift = current_encode_type[3] # На сколько сдвигать указатель при обработке.
                 
                # В идеале менять порядок следования элементов и их запоминать,
                #  но мы обойдемся тем, что будем обрезать по максимальному.
                #
                last_n_max = max(a[offset:offset + n])

                if (offset + n <= list_size) and (last_n_max <= th):

                    tmp = a[offset]
                    for i in range(1, n): 
                        tmp |= (a[offset + i] << (shift * i))

                    res.append(tmp | code)
                    offset += n
                    break

        return res
    

    def decode(self, a):
        """
        Метод класса, позволяюзий ДЕКОДИРОВАТЬ
        список элементов a, по принципу Simple9.
        """        
        
        res = []
        for cur_num in a:

            code = cur_num & 0xf0000000 # cur_num & (1111 0000 0000 ... 0000) - маска для типа SimpleX.
            data = cur_num & 0x0fffffff # cur_num & (0000 1111 1111 ... 1111) - маска для элементов.
            info = self.decode_type[code]

            n     = info[0] # Кол-во чисел которые могут поместиться на регистр.
            bit   = info[1] # Соответствующая бинарная последовательность 1-иц.
            shift = info[2] # На сколько сдвигать указатель при обработке.

            for i in range(n):
                res.append(data & bit)            
                data >>= shift

        return res

In [10]:
# Main:
#
all_data = []
for key, lizt in reversed_index.items():
    
    # Меняем структуру хранения обратных индексов из словаря в список - просто последовательность чисел.
    # Запись идет в следующем формате:
    #  docid  ->  N (lenght of list)  ->  0-st elem in list  ->  1-st of dif_list  ->  ...  ->  N-st of dif_list  ->  docid  ->  ...
    #  (dif_list = differentiated list)
    #
    tmp_list = deepcopy(lizt)
    for i in range(len(tmp_list)-1, 0, -1):
        tmp_list[i] = tmp_list[i] - tmp_list[i-1]
    batch = [key] + [len(tmp_list)] + tmp_list
    all_data.extend(batch)

In [11]:
%%time
# Main:
#
compresser = Simple9_encoder_decoder()        # Создаем объект класса Simple9.
compressed_data = compresser.encode(all_data) # Сжимаем.

CPU times: user 12.5 s, sys: 136 ms, total: 12.6 s
Wall time: 12.6 s


In [12]:
# ... передача сжатых данных по сети ...

In [13]:
%%time
uncompressed_data = compresser.decode(compressed_data) # Декодируем данные обратно.

CPU times: user 1.85 s, sys: 123 ms, total: 1.97 s
Wall time: 2.01 s


In [14]:
%%time
assert(len(all_data) == len(uncompressed_data)) # Проверяем, что все корректно отработало.

for i in range(len(all_data)):
    assert(all_data[i] == uncompressed_data[i]) # Проверяем, что все корректно отработало.

CPU times: user 837 ms, sys: 0 ns, total: 837 ms
Wall time: 842 ms


In [15]:
%%time
parse_state  = 0
new_reversed_index = defaultdict(list)

for elem in uncompressed_data: # Формируем словарь обратных индексов по данной последовательности чисел.
    
    if   parse_state == 0: # Когда текущий elem - termid.
        key = elem
        parse_state = 1
    
    elif parse_state == 1: # Когда текущий elem - lenght of the next list.
        list_len = elem        
        current_list = []
        parse_state = 2
        
    elif parse_state == 2: # Когда текущий elem - list of docid.
        list_len -= 1
        if len(current_list):
            current_list.append(elem + current_list[-1])        
        else:
            current_list.append(elem)
        if list_len == 0:
            parse_state = 0            
            new_reversed_index[key] = deepcopy(current_list)            
    
    else:
        raise "error7"

CPU times: user 7.31 s, sys: 124 ms, total: 7.44 s
Wall time: 7.46 s


In [16]:
assert(len(reversed_index) == len(new_reversed_index)) # Проверяем, что все корректно отработало.

for i in range(len(reversed_index)):    
    assert(reversed_index[i] == new_reversed_index[i]) # Проверяем, что все корректно отработало.

In [17]:
# Все проверки прошли, удалим старый список, чтобы далее фигурировало название получше.
#
reversed_index = new_reversed_index

In [18]:
def search(query):
    """
    Основная функция.
    
    input: запрос заданный в виде бульевого запроса
    
    output: список сайтов, удовлетворяющих ему.
    """
    
    global docid_to_url # Чтобы в конце по docid получить url'ы.

    def get_formula(query):    
        """
        Вспомогательная функция.
        
        input: строка - запрос в формате бульевого запроса.
        
        output: список строк - где каждый объект
         либо операция,
         либо операнд,
         либо скобка
         в бульевом запросе.
        """

        lizt, buf = [], ""

        for c in query.lower():

            if not c.isalpha():

                if buf.strip():
                    lizt.append(buf)

                if c.strip():
                    lizt.append(c)
                buf = ""

            else:    
                buf += c
        if buf.strip():
            lizt.append(buf)  

        return lizt    

    
    def get_polish_notation(formula):
        """
        Вспомогательная функция.
        
        Input: список строк - выход функции get_formula(query) (инфексная запись формулы)
        
        Output: список строк - та же формула, но в порядке обратной польской записи.
        """

        prior = {'!': 3, '&': 2, '|': 1, '(': 0}
        polskRecord = []
        stack = []

        def condition(stack, elem):

            if len(stack) > 0:
                if prior[stack[-1]] >= prior[elem]:
                    return True
            return False


        for elem in formula:

            if elem == '(':
                stack.append(elem)

            elif elem == ')':
                while stack[-1] != '(':
                    polskRecord.append(stack[-1])
                    del stack[-1]
                del stack[-1]

            elif elem in prior.keys():
                while condition(stack, elem):            
                    polskRecord.append(stack[-1])
                    del stack[-1]
                stack.append(elem)

            else:
                polskRecord.append(elem)

        polskRecord.extend(stack[::-1])

        return polskRecord
    


    class getPtr:
        """
        Класс функтор, который позволяет
         получить указатель-адрес вершины дерева.
        """
        def __init__(self):     
            """
            Чтобы не было разных вершин с одним адресом.
            """
            self.all_ptr = set()


        def __call__(self):        
            """
            При вывозове, возвращает 
             новый уникальный указатель.
            """
            tmp = random.randint(0, 10000)

            while tmp in self.all_ptr:
                tmp = random.randint(0, 10000)
            self.all_ptr.add(tmp)

            return tmp
        


    get_ptr = getPtr()
    binTree = dict()

     
    ###
    
    ##
   
    ##
    
    #
    def find_intersection_by_streaming_passage(bin_tree, head_ptr, reversed_index, term_to_id):
  
        def bin_search(arr, key, l=0, r=-100):
            """
            Функция бинарного поиска,
             позволяет при потоковом обходе
             быстрее находить следующий id в списке в листе,
             ограниченный сверху docId.
            """

            if r == -100:
                r = len(arr) - 1

            if l == len(arr) - 1:
                return len(arr) - 1

            if l > r:
                return l

            m = trunc((r + l) / 2)    

            if   arr[m] > key:
                return bin_search(arr, key, l,   m-1)

            elif arr[m] < key:
                return bin_search(arr, key, m+1, r)

            else:
                return m               

        docId = -1 # Главный счётчик текущего инедкса.
        res = []

        def _run(cur_ptr, docId):
            """
            Функция, позволяющая 
             произвоть потоковый проход
             по дереву рекурсивно.
            """

            cur_node = bin_tree[cur_ptr]

            if cur_node[0] == 'BIN_OPERATION':
                if   cur_node[1] == '&':
                    return max(_run(cur_node[2], docId),
                               _run(cur_node[3], docId))

                elif cur_node[1] == '|':
                    return min(_run(cur_node[2], docId),
                               _run(cur_node[3], docId))

                else:
                    raise "error1"

            elif cur_node[0] == 'UN_OPERATION':
                if cur_node[1] == '!':
                    tmp = _run(cur_node[2], docId)
                    if tmp == docId:
                        return tmp+1
                    else:
                        return docId
                else:
                    raise "error3"

            elif cur_node[0] == 'OPERAND':
                doc_list = reversed_index[cur_node[1]]

                cur_node[2] = bin_search(doc_list, docId, l=cur_node[2])

                if   cur_node[2] == len(doc_list)-1:
                    return float("inf")

                elif doc_list[cur_node[2]] >= docId:
                    return doc_list[cur_node[2]]

                else:
                    print(doc_list)
                    print(cur_node)
                    print(docId)
                    raise "error4"

            else:
                raise "error5"                    


        while docId < max_docId:

            query_result = _run(head_ptr, docId)        
            if docId == query_result:
                res.append(docId)
                docId += 1
            else:
                if docId < query_result:
                    docId = query_result
                else:
                    raise "error6"

        return res     
            
        
        
    def make_node(elem, stack, this_type):
        """
        Вспомогательная функция, которая формирует дерево.
        """
        ptr = get_ptr()

        if this_type == 'OPERAND':
            node = {ptr: ['OPERAND', elem, 0]}

        elif this_type == 'UN_OPERATION':
            node = {ptr: ['UN_OPERATION', elem, stack[-1]]}
            del stack[-1]

        elif this_type == 'BIN_OPERATION':
            node = {ptr: ['BIN_OPERATION', elem, stack[-2], stack[-1]]}
            del stack[-2:]

        else:
            raise "error"

        stack.append(ptr)
        binTree.update(node)

        return stack 
    
    # Основоне тело функции search:
    
    # Переводим запрос в список смысловых елементов (слова, скобки, символы &/|/!).
    #
    formula = get_formula(query) 
    
    # Первеодим формулу в польскую запись.
    #
    polishRecord = get_polish_notation(formula)
    
    # Формализуем множество используемых символов-операций.
    #
    operations = {'&', '|', '!'}
    stack = []

    # Итерируемся по формуле, записанной в польской записи
    #  и формируем дерево бинарного поиска.
    #
    for elem in polishRecord:

        if elem in {'&', '|'}:
            stack = make_node(elem, stack, 'BIN_OPERATION')
        elif elem == '!':
            stack = make_node(elem, stack, 'UN_OPERATION')        
        else:
            stack = make_node(term_to_id[elem], stack, 'OPERAND')
            
    # В стеке, находится корень.
    #
    head_ptr = stack[0]
    
    # Запоминаем максимульное значние индекса url'ов.
    #
    max_docId = max(url_to_docid.values())

    # Получаем список из docid, сайтов удовлетворяющих запросу.
    #
    list_of_docid = find_intersection_by_streaming_passage(binTree, head_ptr, reversed_index, term_to_id)
    
    # По docid'шникам получаем сами url'ы.
    #
    needed_urls = []
    for docid in list_of_docid:
        needed_urls.append(docid_to_url[docid])
        
    return needed_urls

In [19]:
# Main:
#
query = "власти & ( бельгии | парижа ) & ! теракт"

In [20]:
# Main:
#
search(query)

['http://lenta.ru/articles/2009/12/30/finalgermany/',
 'http://lenta.ru/articles/2013/04/02/zapretnyplod/',
 'http://lenta.ru/news/2006/07/05/paris',
 'http://lenta.ru/news/2006/08/10/plot2/',
 'http://lenta.ru/articles/2010/11/22/lebannon/',
 'http://lenta.ru/news/2015/08/14/mts_vimpelkom_otkati/',
 'http://lenta.ru/articles/2004/08/21/ruslan/']